In [ ]:
# TODO
%run D:\codes\confg

In [1]:
"""Arome Model read-in Functions
Main functions:
    - read_2D_variables_AROME()  # cheated temperature to 0.5m during night
    - read_3D_variables_AROME() # all heights
    - read_timeSeries_AROME() # interpolated to 2m

"""
import glob
import os

import pandas as pd
import xarray as xr
import numpy as np
import sys
np.set_printoptions(threshold=sys.maxsize)

C:\Users\cosma\anaconda3\envs\myenv\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\cosma\anaconda3\envs\myenv\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
def read_2D_variables_AROME(variableList, lon, lat, slice_lat_lon=False):
    """ WITH the sel Method
    Read all the 2D variables (single netcdf per variable) and merge them

    :param variableList: List of the selected variables
    :param lon: Longitude of the MOMAA station
    :param lat: Latitude of the MOMAA station
    :param slice_lat_lon: Method for selecting latitude and longitude ('nearest' for nearest neighbor, None for exact match)
    :return: Merged DataFrame with all the variables
    """
    datasets = []  # List to hold the interpolated datasets for each variable

    for variable in variableList:
        file_path = os.path.join(dir_2D_AROME, f"AROME_Geosphere_20171015T1200Z_CAP02_2D_30min_1km_best_{variable}.nc")

        ds = xr.open_dataset(file_path)

        # Use no method if lat or lon are slice objects
        if slice_lat_lon:
            ds = ds.sel(longitude=lon, latitude=lat).isel(time=slice(4, None))
        else:
            ds = ds.sel(longitude=lon, latitude=lat, method="nearest").isel(time=slice(4, None))

        for var, units in variables_units_2D_AROME.items():
            if var in ds:
                ds[var].attrs['units'] = units

        ds_quantified = ds.metpy.quantify()
        datasets.append(ds_quantified)

    return xr.merge(datasets, join="exact")

In [3]:
def read_3D_variables_AROME(variables, method, lon, lat, slice_lat_lon=False, level=None, time=None):
    datasets = []  # List to hold datasets for each variable

    for var in variables:
        # Construct the file path and open the dataset
        file_path = os.path.join(dir_3D_AROME, f"AROME_Geosphere_20171015T1200Z_CAP02_3D_30min_1km_best_{var}.nc")
        ds = xr.open_dataset(file_path)

        if time is None:
            time_start = pd.to_datetime('2017-10-15 14:00:00',
                                        format='%Y-%m-%d %H:%M:%S')
            time_end = pd.to_datetime('2017-10-16 12:00:00',
                                      format='%Y-%m-%d %H:%M:%S')
            time = pd.date_range(start=time_start, end=time_end, freq='30min')

        # Select or interpolate the dataset based on the method
        ds_selected = ds.isel(record=30, x=332, y=214) #IMST
            
        # Update variable units
        for variable, units in variables_units_3D_AROME.items():
            if variable in ds_selected:
                ds_selected[variable].attrs['units'] = units

        # Quantify the dataset and append to the list
        datasets.append(ds_selected.metpy.quantify())

    # Merge all datasets
    return xr.merge(datasets, join="exact")

In [4]:
def read_timeSeries_AROME(location):
    """The Timeseries is a direct AROME model output which holds all variables (see Data_structure.md) for a specific
    location
    ::param location: is the selected location
    """
    pattern = f"AROME_Geosphere_20171015T1200Z_{location}_timeseries_40s_*.nc"
    final_path_pattern = os.path.join(dir_timeseries_AROME, pattern)

    # Use glob to find files that match the pattern
    matching_files = glob.glob(final_path_pattern)

    # Assuming there's only one match per location, open the dataset
    if matching_files:
        return xr.open_dataset(matching_files[0])
    else:
        raise FileNotFoundError(f"No files found for location {location}")

In [5]:
# # to test which gridpoint to take
# my_variable_list = ["p", "q", "th", "u", "v", "z"]

# datasets = []  # List to hold datasets for each variable

# for var in my_variable_list:
#     # Construct the file path and open the dataset
#     file_path = os.path.join(dir_3D_AROME, f"AROME_Geosphere_20171015T1200Z_CAP02_3D_30min_1km_best_{var}.nc")
#     ds = xr.open_dataset(file_path)

#     ds_selected = ds.isel(record=30, x=398, y=215) #referenzwert ibk
    

# ds.isel(record=30, x=332, y=214).latitude.values #logitude = 10.725, latitude = 47.235 for Imst at (47.2419° N, 10.7218° E)


In [6]:
# read_3D_variables_AROME(variables=my_variable_list, method='sel', lon=15.1, lat=47.35, time=30) 
# #angegebener wert für lon und lat ist komplett irrelevant für output
